In [ ]:
import os
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from monai.metrics import DiceMetric, HausdorffDistanceMetric
from monai.transforms import AsDiscrete
from monai.utils import evenly_divisible_all_gather
from tqdm import tqdm
import torch

labels_path = ""
predictions_path = ""

label_files = [f for f in os.listdir(labels_path) if f.endswith(".nii.gz")]
prediction_files = [f for f in os.listdir(predictions_path) if f.endswith(".nii.gz")]

label_files.sort()
prediction_files.sort()

results = []

dice_metric = DiceMetric(include_background=False, reduction="mean_batch")
hd95_metric = HausdorffDistanceMetric(
    include_background=False, reduction="mean", percentile=95
)


def compute_volumetric_similarity(label_img, prediction_img):
    # Volumetric Similarity (VS) = 1 - |V_pred - V_label| / (V_pred + V_label)
    label_volume = np.sum(label_img > 0)
    prediction_volume = np.sum(prediction_img > 0)
    if label_volume + prediction_volume == 0:
        return 1.0
    return 1 - abs(prediction_volume - label_volume) / (
        prediction_volume + label_volume
    )


def compute_nsd(label_img, prediction_img, threshold=1.0):
    # Normalized Surface Dice (NSD)
    from scipy.ndimage import distance_transform_edt

    label_surface = np.where(label_img > 0, 1, 0)
    prediction_surface = np.where(prediction_img > 0, 1, 0)

    label_dist = distance_transform_edt(1 - label_surface)
    prediction_dist = distance_transform_edt(1 - prediction_surface)

    label_surface_pts = np.sum((label_dist <= threshold) & (label_surface == 1))
    prediction_surface_pts = np.sum(
        (prediction_dist <= threshold) & (prediction_surface == 1)
    )

    total_surface_pts = label_surface_pts + prediction_surface_pts
    if total_surface_pts == 0:
        return 1.0
    return (2 * label_surface_pts) / total_surface_pts


def compute_precision_recall_iou(label_img, prediction_img):
    TP = np.sum((label_img == 1) & (prediction_img == 1))
    FP = np.sum((label_img == 0) & (prediction_img == 1))
    FN = np.sum((label_img == 1) & (prediction_img == 0))
    TN = np.sum(
        (label_img == 0) & (prediction_img == 0)
    )  # True Negatives (optional, for completeness)

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0

    iou = TP / (TP + FP + FN) if (TP + FP + FN) > 0 else 0

    return precision, recall, iou


for label_file, prediction_file in tqdm(
    zip(label_files, prediction_files), total=len(label_files), desc="Processing"
):
    label_path = os.path.join(labels_path, label_file)
    prediction_path = os.path.join(predictions_path, prediction_file)

    label_img = nib.load(label_path).get_fdata()
    prediction_img = nib.load(prediction_path).get_fdata()

    label_img = torch.from_numpy(label_img).unsqueeze(0).unsqueeze(0)
    prediction_img = torch.from_numpy(prediction_img).unsqueeze(0).unsqueeze(0)
    if torch.sum(prediction_img)==0:
        print(prediction_file)
    dice_value = dice_metric(y_pred=prediction_img, y=label_img).item()

    hd95_value = hd95_metric(y_pred=prediction_img, y=label_img).item()

    vs_value = compute_volumetric_similarity(
        label_img.squeeze().numpy(), prediction_img.squeeze().numpy()
    )
    precision_value, recall_value, iou_value = compute_precision_recall_iou(
        label_img.squeeze().numpy(), prediction_img.squeeze().numpy()
    )
    results.append(
        {
            "Name": label_file,
            "Dice": dice_value,
            "HD95": hd95_value,
            "VS": vs_value,
            "Precision": precision_value,
            "Recall": recall_value,
            "IoU": iou_value,
        }
    )

df = pd.DataFrame(results)

print(df.describe())
df.to_csv(
    os.path.join(os.path.dirname(predictions_path), os.path.basename(predictions_path)+".csv"),
    index=False,
)
metrics = ["Dice", "HD95", "VS", "Precision", "Recall", "IoU"]
fig, axes = plt.subplots(2, 3, figsize=(20, 10), dpi=120)
axes = axes.flatten()

for i, metric in enumerate(metrics):
    axes[i].hist(df[metric].dropna(), bins=30, edgecolor="black")
    axes[i].set_title(f"{metric} Distribution")
    axes[i].set_xlabel(metric)
    axes[i].set_ylabel("Frequency")
    axes[i].grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import os
from glob import glob
from scipy.ndimage import rotate
import matplotlib.patches as patches

def compute_dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection) / (np.sum(y_true) + np.sum(y_pred) + 1e-8)

def calculate_dice_score(pred, target):
    smooth = 1e-5
    intersection = np.sum(pred * target)
    return (2. * intersection + smooth) / (np.sum(pred) + np.sum(target) + smooth)

def add_zoom_to_image_with_pred(image, label_mask, pred_mask, zoom_region, zoom_position, ax, alpha=0.35):
    result = image.copy()
    
    zoom_area = image[zoom_region[0]:zoom_region[1], zoom_region[2]:zoom_region[3]]
    zoom_label = label_mask[zoom_region[0]:zoom_region[1], zoom_region[2]:zoom_region[3]]
    zoom_pred = pred_mask[zoom_region[0]:zoom_region[1], zoom_region[2]:zoom_region[3]]
    
    zoom_height = zoom_position[1] - zoom_position[0]
    zoom_width = zoom_position[3] - zoom_position[2]
    
    from scipy.ndimage import zoom
    zoom_factors = (zoom_height/zoom_area.shape[0], zoom_width/zoom_area.shape[1])
    zoomed = zoom(zoom_area, zoom_factors, order=1)
    zoomed_label = zoom(zoom_label, zoom_factors, order=0)
    zoomed_pred = zoom(zoom_pred, zoom_factors, order=0)
    
    result[zoom_position[0]:zoom_position[1], zoom_position[2]:zoom_position[3]] = zoomed
    
    ax.imshow(result, cmap='gray')
    
    label_mask = np.ma.masked_where(label_mask == 0, label_mask)
    pred_mask = np.ma.masked_where(pred_mask == 0, pred_mask)
    ax.imshow(label_mask, alpha=alpha, cmap='winter')
    ax.imshow(pred_mask, alpha=alpha, cmap='autumn')
    
    zoomed_label_full = np.zeros_like(result)
    zoomed_pred_full = np.zeros_like(result)
    zoomed_label_full[zoom_position[0]:zoom_position[1], zoom_position[2]:zoom_position[3]] = zoomed_label
    zoomed_pred_full[zoom_position[0]:zoom_position[1], zoom_position[2]:zoom_position[3]] = zoomed_pred
    
    zoomed_label_full = np.ma.masked_where(zoomed_label_full == 0, zoomed_label_full)
    zoomed_pred_full = np.ma.masked_where(zoomed_pred_full == 0, zoomed_pred_full)
    ax.imshow(zoomed_label_full, alpha=alpha, cmap='winter')
    ax.imshow(zoomed_pred_full, alpha=alpha, cmap='autumn')
    
    rect_original = patches.Rectangle((zoom_region[2], zoom_region[0]), 
                                    zoom_region[3]-zoom_region[2], 
                                    zoom_region[1]-zoom_region[0], 
                                    linewidth=2, edgecolor='w', facecolor='none')
    rect_zoomed = patches.Rectangle((zoom_position[2], zoom_position[0]), 
                                   zoom_position[3]-zoom_position[2], 
                                   zoom_position[1]-zoom_position[0], 
                                   linewidth=2, edgecolor='w', facecolor='none')
    ax.add_patch(rect_original)
    ax.add_patch(rect_zoomed)
    
    ax.plot([zoom_region[2], zoom_position[2]], 
            [zoom_region[0], zoom_position[0]], 
            'w--', linewidth=2)
    ax.plot([zoom_region[3], zoom_position[3]], 
            [zoom_region[1], zoom_position[1]], 
            'w--', linewidth=2)
    
    return result
def add_zoom_to_image(image, mask, zoom_region, zoom_position, ax, alpha=0.35, cmap='winter'):

    result = image.copy()
    
    zoom_area = image[zoom_region[0]:zoom_region[1], zoom_region[2]:zoom_region[3]]
    zoom_mask = mask[zoom_region[0]:zoom_region[1], zoom_region[2]:zoom_region[3]]
    
    zoom_height = zoom_position[1] - zoom_position[0]
    zoom_width = zoom_position[3] - zoom_position[2]
    
    from scipy.ndimage import zoom
    zoom_factors = (zoom_height/zoom_area.shape[0], zoom_width/zoom_area.shape[1])
    zoomed = zoom(zoom_area, zoom_factors, order=1)
    zoomed_mask = zoom(zoom_mask, zoom_factors, order=0)  # 使用order=0保持标签值
    
    result[zoom_position[0]:zoom_position[1], zoom_position[2]:zoom_position[3]] = zoomed
    
    ax.imshow(result, cmap='gray')
    mask = np.ma.masked_where(mask == 0, mask)
    ax.imshow(mask, alpha=alpha, cmap=cmap)
    
    zoomed_mask_full = np.zeros_like(result)
    zoomed_mask_full[zoom_position[0]:zoom_position[1], zoom_position[2]:zoom_position[3]] = zoomed_mask
    zoomed_mask_full = np.ma.masked_where(zoomed_mask_full == 0, zoomed_mask_full)
    ax.imshow(zoomed_mask_full, alpha=alpha, cmap=cmap)
    
    rect_original = patches.Rectangle((zoom_region[2], zoom_region[0]), 
                                    zoom_region[3]-zoom_region[2], 
                                    zoom_region[1]-zoom_region[0], 
                                    linewidth=2, edgecolor='w', facecolor='none')
    rect_zoomed = patches.Rectangle((zoom_position[2], zoom_position[0]), 
                                   zoom_position[3]-zoom_position[2], 
                                   zoom_position[1]-zoom_position[0], 
                                   linewidth=2, edgecolor='w', facecolor='none')
    ax.add_patch(rect_original)
    ax.add_patch(rect_zoomed)
    
    ax.plot([zoom_region[2], zoom_position[2]], 
            [zoom_region[0], zoom_position[0]], 
            'w--', linewidth=2)
    ax.plot([zoom_region[3], zoom_position[3]], 
            [zoom_region[1], zoom_position[1]], 
            'w--', linewidth=2)
    
    return result
def visualize_segmentation_range(original_path, label_path, pred_folders, slice_start, slice_end, save_path, 
                               alpha=0.5, rotate_angle=0, 
                               zoom_region=(100, 150, 50, 100),
                               zoom_position=(300, 400, 50, 150)):
    os.makedirs(save_path, exist_ok=True)
    
    for slice_num in range(slice_start, slice_end + 1):
        print(f"Processing slice {slice_num}")
        orig_img = nib.load(original_path)
        orig_slice = orig_img.get_fdata()[:,:,slice_num]
        label_img = nib.load(label_path)
        label_slice = label_img.get_fdata()[:,:,slice_num]
        
        orig_slice_normalized = (orig_slice - orig_slice.min()) / (orig_slice.max() - orig_slice.min())
        contrast, brightness = 0.5, 0.65
        orig_slice_enhanced = np.clip(contrast * orig_slice_normalized + brightness, 0, 1)
        
        if rotate_angle != 0:
            orig_slice_enhanced = rotate(orig_slice_enhanced, rotate_angle, order=1)
            label_slice = rotate(label_slice, rotate_angle, order=0)
        
        base_name = os.path.splitext(os.path.splitext(os.path.basename(label_path))[0])[0]
        dpi = 300
        fontsize = 22
        plt.rcParams['figure.dpi'] = dpi
        plt.rcParams['font.size'] = fontsize

        fig, ax = plt.subplots(figsize=(8, 8))
        orig_with_zoom = add_zoom_to_image(orig_slice_enhanced, np.zeros_like(label_slice),
                                         zoom_region, zoom_position, ax, alpha=alpha)
        ax.text(0.98, 0.98, 'Original Image', 
                fontsize=fontsize,
                horizontalalignment='right',
                verticalalignment='top',
                transform=ax.transAxes,
                color='white',
                bbox=dict(facecolor='black', alpha=0.5))
        plt.axis('off')
        plt.savefig(os.path.join(save_path, f'original_slice_{slice_num}.png'), 
                    bbox_inches='tight', pad_inches=0, dpi=dpi)
        plt.close()

        fig, ax = plt.subplots(figsize=(8, 8))
        orig_with_zoom = add_zoom_to_image(orig_slice_enhanced, label_slice,
                                         zoom_region, zoom_position, ax, 
                                         alpha=alpha, cmap='winter')
        ax.text(0.98, 0.98, 'Ground Truth', 
                fontsize=fontsize,
                horizontalalignment='right',
                verticalalignment='top',
                transform=ax.transAxes,
                color='white',
                bbox=dict(facecolor='black', alpha=0.5))
        plt.axis('off')
        plt.savefig(os.path.join(save_path, f'ground_truth_slice_{slice_num}.png'), 
                    bbox_inches='tight', pad_inches=0, dpi=dpi)
        plt.close()

        for folder in pred_folders:
            pred_path = glob(os.path.join(folder, f"*{base_name}*.nii.gz"))
            model_name = os.path.basename(folder).replace('_NormalLoss', '').replace('_SelfLoss', '')
            loss_type = 'NormalLoss' if 'NormalLoss' in folder else 'SelfLoss'
            
            if pred_path:
                pred_img = nib.load(pred_path[0])
                pred_slice = pred_img.get_fdata()[:,:,slice_num]
                if rotate_angle != 0:
                    pred_slice = rotate(pred_slice, rotate_angle, order=0)
                
                dice_score = compute_dice_coefficient(label_slice, pred_slice)
                
                fig, ax = plt.subplots(figsize=(8, 8))
                add_zoom_to_image_with_pred(orig_slice_enhanced, label_slice, pred_slice,
                                        zoom_region, zoom_position, ax, alpha=alpha)
                
                ax.text(0.98, 0.98, f'{model_name}_FR', 
                        fontsize=fontsize,
                        horizontalalignment='right',
                        verticalalignment='top',
                        transform=ax.transAxes,
                        color='white',
                        bbox=dict(facecolor='black', alpha=0.5))
                
                ax.text(0.98, 0.90, f'Dice: {dice_score*100:.2f}%', 
                        fontsize=fontsize-5,
                        horizontalalignment='right',
                        verticalalignment='top',
                        transform=ax.transAxes,
                        color='white',
                        bbox=dict(facecolor='black', alpha=0.5))
                
                plt.axis('off')
                plt.savefig(os.path.join(save_path, f'{model_name}_{loss_type}_slice_{slice_num}.png'), 
                            bbox_inches='tight', pad_inches=0, dpi=dpi)
                plt.close()
            else:
                print(f"Warning: No matching prediction found for {base_name} in {folder}")

patient_num = "0397"
original_path = f""
label_path = f""
save_path = f""

base_pred_folders = [
]
Loss = "NormalLoss"
pred_folders = [folder + Loss for folder in base_pred_folders]

slice_start = 58
slice_end = 58
alpha = 0.35
rotate_angle = -90

zoom_region = (123, 183, 70, 130)
zoom_position = (8, 113, 155, 260)

visualize_segmentation_range(
    original_path, 
    label_path, 
    pred_folders, 
    slice_start, 
    slice_end, 
    save_path,
    alpha, 
    rotate_angle,
    zoom_region,
    zoom_position
)


In [ ]:
import pandas as pd

df = pd.read_csv('your_file.csv') 

df = df.drop('HD95', axis=1)

means = df.drop('Name', axis=1).mean()

print(f"Dice: {means['Dice']:.4f}")
print(f"VS: {means['VS']:.4f}")
print(f"Precision: {means['Precision']:.4f}")
print(f"Recall: {means['Recall']:.4f}")
print(f"IoU: {means['IoU']:.4f}")

print(df.drop('Name', axis=1).describe())

df.to_csv('processed_file.csv', index=False)

Dice: 0.7792
VS: 0.8675
Precision: 0.8034
Recall: 0.7661
IoU: 0.6815
            Dice         VS  Precision     Recall        IoU
count  25.000000  25.000000  25.000000  25.000000  25.000000
mean    0.779152   0.867467   0.803367   0.766147   0.681507
std     0.244524   0.256190   0.257001   0.248737   0.228527
min     0.000000   0.000000   0.000000   0.000000   0.000000
25%     0.784792   0.875179   0.843805   0.743465   0.645809
50%     0.862046   0.961682   0.896721   0.852076   0.757540
75%     0.888393   0.982550   0.920034   0.893545   0.799198
max     0.943901   0.999855   0.963051   0.941564   0.893762
